In [1]:
import time
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
from pipeline import create_pipeline
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [2]:
# 전처리
# Get Dataset
files={
    'CICI':'/home/irteam/junghye-dcloud-dir/MLAC/new_data/CICI.csv',
    'UNSW':'/home/irteam/junghye-dcloud-dir/MLAC/new_data/UNSW.csv'
}

data = pd.read_csv(files['CICI'])
data=data[np.isfinite(data).all(1)]

#multiclass_labels_1=data['nist_category']
multiclass_labels_2=data['attack_category']

data=data.drop(labels=['label','attack_category','nist_category'],axis=1)

In [3]:
# Define Models
models = []
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('ABoost', AdaBoostClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('MLP', MLPClassifier()))

In [4]:
df=pd.DataFrame(columns=['name','acc','f1_mi','f1_ma','f1_we','rc_mi','rc_ma','rc_we']+\
                 ['pc_mi','pc_ma','pc_we'])
eval_path='/home/irteam/junghye-dcloud-dir/MLAC/evaluation'
confusion_path='/home/irteam/junghye-dcloud-dir/MLAC/confusion_matrix/CICI_16class'
cnt=0

In [5]:
X_train,X_test,y_train,y_test=train_test_split(data,multiclass_labels_2,test_size=0.3, shuffle=True, stratify=multiclass_labels_2, random_state=34)

In [6]:
# confusion matrix plot
def plot_confusion_matrix(con_mat,labels,title:str,cmap=plt.cm.get_cmap('Blues'),normalize=False):
    plt.figure(figsize=(16,8))
    plt.imshow(con_mat,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks=np.arange(len(labels))
    nlabels=[]
    for k in range(len(con_mat)):
        n=sum(con_mat[k])
        nlabel='{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)

    plt.xticks(marks,labels,rotation=45)
    plt.yticks(marks,nlabels)

    thresh=con_mat.max()/2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}%'.format(con_mat[i, j] * 100 / n), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.show()
    #이미지 저장
    plt.savefig(confusion_path+'/'+title+'.png',facecolor='#eeeeee',edgecolor='blue')
    plt.clf()

In [ ]:
for name, model in models:
    #data_loader()
    # binary classification
    model=create_pipeline(model)
   
    print("training start.."+name)
    model.fit(X_train,y_train)
    
    #evaluation
    print('evalutaion start..')
    y_pred=model.predict(X_test)
    #evaluation result
    model_eval=[]
    model_eval.append(name)
    
    acc = accuracy_score(y_test, y_pred)
    f1_mi = f1_score(y_test, y_pred,average='micro')
    f1_ma = f1_score(y_test, y_pred,average='macro')
    f1_we = f1_score(y_test, y_pred,average='weighted')
    recall_mi = recall_score(y_test, y_pred, average='micro')
    recall_ma = recall_score(y_test, y_pred, average='macro')
    recall_we = recall_score(y_test, y_pred, average='weighted')
    precision_mi = precision_score(y_test, y_pred, average='micro')
    precision_ma = precision_score(y_test, y_pred, average='macro')
    precision_we = precision_score(y_test, y_pred, average='weighted')
    

    model_eval.extend([acc,f1_mi,f1_ma,f1_we,recall_mi,recall_ma,recall_we,precision_mi,precision_ma,precision_we])
    
    #confusion_metrics
    confusion=metrics.confusion_matrix(y_test,y_pred)
    
    plot_confusion_matrix(confusion,labels=['Benign','Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye', 'DoS Slowhttptest', 'DoS slowloris', 'Dos Hulk', 'Exploits', 'FTP-Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration']+\
                                             ['PortScan', 'Reconnaissance', 'SSH-Patator', 'Shellcode', 'Web Attack – Brute Force', 'Web Attack – Sql Injection', 'Web Attack – XSS', 'Worms'],title=name)
       


    print(f'name:{name},acc:{acc},f1_score:{f1_mi},{f1_ma},{f1_we},recall:{recall_mi},{recall_ma},{recall_we},precision:{precision_mi},{precision_ma},{precision_we}')
    df.loc[cnt]=model_eval

    cnt+=1
    

df.to_csv(os.path.join(eval_path,'CICI_25class.csv'),index=False)

training start..RF
evalutaion start..
name:RF,acc:0.9465881011214597,f1_score:0.9465881011214597,0.3123132167235964,0.9367214753346027,recall:0.9465881011214597,0.2930530809082538,0.9465881011214597,precision:0.9465881011214597,0.35026745914887314,0.9363931593572624
training start..CART
evalutaion start..
name:CART,acc:0.982206851150435,f1_score:0.982206851150435,0.40141846652709007,0.976867589701638,recall:0.982206851150435,0.37145816461098874,0.982206851150435,precision:0.982206851150435,0.5649154220323831,0.9747664827779271
training start..NB
evalutaion start..
name:NB,acc:0.7823978562918114,f1_score:0.7823978562918114,0.5034931857091386,0.8413174854255229,recall:0.7823978562918114,0.8907669264470603,0.7823978562918114,precision:0.7823978562918114,0.47729111019756215,0.934752280643054
training start..LDA
evalutaion start..
name:LDA,acc:0.9461600394373075,f1_score:0.9461600394373075,0.4774904168787731,0.9534724135036096,recall:0.9461600394373075,0.6992959019101015,0.9461600394373075,

In [ ]:
from PIL import Image

#파일명
img_files=os.listdir(confusion_path)

images=[] # 이미지 리스트
for img_file in img_files:
    img=Image.open(img_file)
    img.resize((400,300))
    images.append(img)

# 크기
width,height=images[0].size
new_width=width*3
new_height=height*3
new_img=Image.new('RGB',(new_width,new_height))

#이미지 합치기
x_offset=0
y_offset=0
for img in images:
    new_img.paste(img,(x_offset,y_offset))
    x_offset+=width
    if x_offset==new_width:
        x_offset=0
        y_offset+=height


new_img.save(confusion_path+'merged_image.jpg')